In [1]:
import copy
import time

program = open("21.txt").readlines()
program=[[int(y) if y.isnumeric() else y for y in x.strip().split(' ')] for x in program]
ip_reg=program[0][1] 
program=program[1:]

#print original syntax
for i, l in enumerate(program):
    print(i,l)

    



0 ['seti', 123, 0, 3]
1 ['bani', 3, 456, 3]
2 ['eqri', 3, 72, 3]
3 ['addr', 3, 4, 4]
4 ['seti', 0, 0, 4]
5 ['seti', 0, 2, 3]
6 ['bori', 3, 65536, 2]
7 ['seti', 1397714, 1, 3]
8 ['bani', 2, 255, 5]
9 ['addr', 3, 5, 3]
10 ['bani', 3, 16777215, 3]
11 ['muli', 3, 65899, 3]
12 ['bani', 3, 16777215, 3]
13 ['gtir', 256, 2, 5]
14 ['addr', 5, 4, 4]
15 ['addi', 4, 1, 4]
16 ['seti', 27, 6, 4]
17 ['seti', 0, 6, 5]
18 ['addi', 5, 1, 1]
19 ['muli', 1, 256, 1]
20 ['gtrr', 1, 2, 1]
21 ['addr', 1, 4, 4]
22 ['addi', 4, 1, 4]
23 ['seti', 25, 2, 4]
24 ['addi', 5, 1, 5]
25 ['seti', 17, 0, 4]
26 ['setr', 5, 7, 2]
27 ['seti', 7, 4, 4]
28 ['eqrr', 3, 0, 5]
29 ['addr', 5, 4, 4]
30 ['seti', 5, 8, 4]


In [2]:
#convert funny syntax to more readable
def pretty(l):
    r=""

    d={}
    d["addr"]="(C)=(A)+(B)"
    d["addi"]="(C)=(A)+B"
    d["mulr"]="(C)=(A)*(B)"
    d["muli"]="(C)=(A)*B"
    d["banr"]="(C)=(A)&(B)"
    d["bani"]="(C)=(A)&B"
    d["borr"]="(C)=(A)|(B)"
    d["bori"]="(C)=(A)|B"
    d["setr"]="(C)=(A)"
    d["seti"]="(C)=A"
    d["gtir"]="(C)=A>(B)"
    d["gtri"]="(C)=(A)>B"
    d["gtrr"]="(C)=(A)>(B)"
    d["eqir"]="(C)=A==(B)"
    d["eqri"]="(C)=(A)==B"
    d["eqrr"]="(C)=(A)==(B)"

    r=d[l[0]]
    r=r.replace('A',str(l[1]))
    r=r.replace('B',str(l[2]))
    r=r.replace('C',str(l[3]))
    for j in range(6):
        if j == ip_reg:
            r=r.replace("(%d)"%(j),"IX")
        else:
            r=r.replace("(%d)"%(j),"R%d"%(j))
    if any([x in r for x in ["==","<",">"]]):
        r+= "   !!!"
    
    return r

for i, l in enumerate(program):
    print("% 6d   %s"%(i,pretty(l)))

     0   R3=123
     1   R3=R3&456
     2   R3=R3==72   !!!
     3   IX=R3+IX
     4   IX=0
     5   R3=0
     6   R2=R3|65536
     7   R3=1397714
     8   R5=R2&255
     9   R3=R3+R5
    10   R3=R3&16777215
    11   R3=R3*65899
    12   R3=R3&16777215
    13   R5=256>R2   !!!
    14   IX=R5+IX
    15   IX=IX+1
    16   IX=27
    17   R5=0
    18   R1=R5+1
    19   R1=R1*256
    20   R1=R1>R2   !!!
    21   IX=R1+IX
    22   IX=IX+1
    23   IX=25
    24   R5=R5+1
    25   IX=17
    26   R2=R5
    27   IX=7
    28   R5=R3==R0   !!!
    29   IX=R5+IX
    30   IX=5


In [3]:
#execute
def execute(p,ix,r0=0,limit=None,part=0):
    opA,opB,opC=range(1,4)
    ip=0
    r=[r0,0,0,0,0,0]
    c=[0 for x in range(len(p))]
    e=0
    pl=len(p)
    m=[{} for y in range(pl)]
    minr3=[]
    while ip < len(p):
        r[ix]=ip
        if ip == 28:
            if part==0:
                print("part1:",r[3])
                break
            else:
                print("part2:",r[3], r[3] in minr3)
                minr3.append(r[3])
        c[ip]+=1
        cmd,opA,opB,opC=p[r[ix]]
        #print(cmd,r[ix])
        if cmd=="addr":
            r[opC]=r[opA]+r[opB]
        elif cmd=="addi":
            r[opC]=r[opA]+opB
        elif cmd=="mulr":
            r[opC]=r[opA]*r[opB]
        elif cmd=="muli":
            r[opC]=r[opA]*opB
        elif cmd=="banr":
            r[opC]=r[opA]&r[opB]
        elif cmd=="bani":
            r[opC]=r[opA]&opB
        elif cmd=="borr":
            r[opC]=r[opA]|r[opB]
        elif cmd=="bori":
            r[opC]=r[opA]|opB
        elif cmd=="setr":
            r[opC]=r[opA]
        elif cmd=="seti":
            r[opC]=opA
        elif cmd=="gtir":
            r[opC]=int(opA>r[opB])
        elif cmd=="gtri":
            r[opC]=int(r[opA]>opB)
        elif cmd=="gtrr":
            r[opC]=int(r[opA]>r[opB])
        elif cmd=="eqir":
            r[opC]=int(opA==r[opB])
        elif cmd=="eqri":
            r[opC]=int(r[opA]==opB)
        elif cmd=="eqrr":
            r[opC]=int(r[opA]==r[opB])
        else:
            assert(0)
        bip=ip
        ip=r[ix]
        jmp= bip!=ip
        ip+=1
        if jmp and ip<len(p):
            if not bip in m[ip]:
                m[ip][bip]=0
            m[ip][bip]+=1
        
        if limit and sum(c)>=limit:
            print("interrupt")
            e=1
            break
    return r,c,m,e

t = time.time()
r,c,m,e=execute(program,ip_reg,0,100000)
dt = time.time()-t
print("execution time %fs"%(dt))


part1: 3909249
execution time 0.001573s


In [4]:
t = time.time()
r,c,m,e=execute(program,ip_reg,0,1020000,1)
dt = time.time()-t
print("execution time %fs"%(dt))


part2: 3909249 False
part2: 16433800 False
part2: 8267081 False
part2: 5607827 False
part2: 11807759 False
interrupt
execution time 0.646258s


In [ ]:
#execute
def execute2(p,ix,r0=0,limit=None):
    opA,opB,opC=range(1,4)
    
    r=[r0,0,0,0,0,0]
    h=[[] for x in range(len(p))]
    e=0
    pl=len(p)
    
    lastip=None
    while r[ix] < len(p):
        lastip=r[ix]
        cmd,opA,opB,opC=p[r[ix]]
        #print(cmd,r[ix])
        if cmd=="addr":
            r[opC]=r[opA]+r[opB]
        elif cmd=="addi":
            r[opC]=r[opA]+opB
        elif cmd=="mulr":
            r[opC]=r[opA]*r[opB]
        elif cmd=="muli":
            r[opC]=r[opA]*opB
        elif cmd=="banr":
            r[opC]=r[opA]&r[opB]
        elif cmd=="bani":
            r[opC]=r[opA]&opB
        elif cmd=="borr":
            r[opC]=r[opA]|r[opB]
        elif cmd=="bori":
            r[opC]=r[opA]|opB
        elif cmd=="setr":
            r[opC]=r[opA]
        elif cmd=="seti":
            r[opC]=opA
        elif cmd=="gtir":
            r[opC]=int(opA>r[opB])
        elif cmd=="gtri":
            r[opC]=int(r[opA]>opB)
        elif cmd=="gtrr":
            r[opC]=int(r[opA]>r[opB])
        elif cmd=="eqir":
            r[opC]=int(opA==r[opB])
        elif cmd=="eqri":
            r[opC]=int(r[opA]==opB)
        elif cmd=="eqrr":
            r[opC]=int(r[opA]==r[opB])
        else:
            assert(0)
        ip=r[ix]
        ip+=1
        if limit and sum(c)>=limit:
            print("interrupt")
            break
    return r,c,f

t = time.time()
r,m,e=execute(program,ip_reg,0,100000)
dt = time.time()-t
print("execution time %fs"%(dt))

In [5]:
#print with execution info
lns=["%02d % 15s"%(i,pretty(inst)) for i,inst in enumerate(program)]
for i in range(len(lns)):
    #lns[i]+="% 3d"%(c[i])
    fnd=0
    for s in m[i].keys():
        #lns[i]+=" %d"%(s)
        lns[i]+=" %d(%d)"%(s,m[i][s])
    jmp=0
    for j in range(len(lns)):
        if i in m[j].keys():
            jmp=1
    if jmp or "IX" in lns[i]:
        lns[i]+=" >>"
            
#Print pretty syntax            
for l in lns:
    print(l)


00          R3=123
01       R3=R3&456
02 R3=R3==72   !!!
03        IX=R3+IX >>
04            IX=0 >>
05            R3=0 3(1)
06     R2=R3|65536 30(5)
07      R3=1397714
08       R5=R2&255 27(10)
09        R3=R3+R5
10  R3=R3&16777215
11     R3=R3*65899
12  R3=R3&16777215
13 R5=256>R2   !!!
14        IX=R5+IX >>
15         IX=IX+1 >>
16           IX=27 14(5) >>
17            R5=0 15(11)
18         R1=R5+1 25(145679)
19       R1=R1*256
20  R1=R1>R2   !!!
21        IX=R1+IX >>
22         IX=IX+1 >>
23           IX=25 21(10) >>
24         R5=R5+1 22(145680)
25           IX=17 >>
26           R2=R5 23(10)
27            IX=7 >>
28 R5=R3==R0   !!! 16(5)
29        IX=R5+IX >>
30            IX=5 >>


In [6]:
def op(r0):
    while 1:
        r3=123       #00
        r3&=456      #01
        if r3==72:   
            break
    r3=0             #05
    while 1:
        r2=r3|65536  #06
        r3=1397714   #07
        r5=r2&255    #08
        r3+=r5       #09
        r3&=16777215 #10
        r3*=65899    #11
        r3&=16777215 #12
        r5=256>r2    #13
        
        if r5:
            r5=0         #17
            r5+=1        #18
            r1*=255      #19
            r1=r1>r2     #20
        
        r5+=1        #24
        
        
        r5=r3==r0    #
        if r5:
            break
        
        
        
        
        
        
        
    

